# Exploratory Data Analysis of Successful Film Trends
## Informing Decision-making for Microsoft's New Movie Studio

# 1. Business Understanding
Microsoft has identified an opportunity to enter the movie industry by establishing a new movie studio. However, as a technology company, they lack expertise in the domain of filmmaking. To make informed decisions about the type of films to produce, Microsoft needs to understand the current landscape of the movie industry.



The primary objective is to explore successful movies and translate the findings into actionable insights. Through exploratory data analysis, insights of the types of films doing well can be generated.These insights will serve as a foundation for determining the types of films the new studio should create to maximize their chances of achieving box office success.

This analysis aims to answer the following key questions:
* What are the top-performing genres in the movie industry?
* Are there any particular demographics or target audiences that have shown a strong preference for certain types of films?
* What is the average budget range for successful movies, and how does it correlate with their box office performance?
* Are there any seasonal or temporal factors influencing the success of films, such as release dates or holiday periods?
* Are there any specific marketing or promotional strategies employed by successful movies that contribute to their box office success?
* How do critical reviews and audience ratings impact the commercial success of movies?

# 2. Data Understanding
To gain a comprehensive understanding of the movie industry and address Microsoft's business problem, several types of data can be collected and analyzed. 
The datasets used for this anaysis contains information from movie sites. The files included  are `'tn.movie_budgets.csv'`, `'bom.movie_gross.csv'`, `'tmdb.movies.csv'` and a database called `'im.db'`.
Information contained in these files includes:
+ Genres
+ Movie Titles
+ Movie Ratings
+ Movie Budgets
+ Domestic and Foreign Gross Income
+ Release dates
+ Runtime Minutes
+ Popularity

## Relevance of this information
* We can be able to understand audience preferences by analyzing the performance of different genres which can help determine which genres are currently popular and have a higher likelihood of success.
* Analyzing ratings can help identify films that have been well-received and positively reviewed, suggesting higher chances of success.
* Analyzing the performance of films across different budget ranges can help identify patterns and determine optimal budget allocation strategies for Microsoft's new movie studio.
* Analyzing the performance of films in different markets can inform decisions regarding international distribution strategies and potential revenue streams.
* Analyzing runtime minutes can help identify patterns regarding film duration and potential audience preferences.
* Analyzing popularity metrics can help gauge the potential success and audience reception of movies.


# 3. Data Preparation

We first import the relevant packages for this analysis.

In [1]:
# importing the relevant packages
import pandas as pd
import numpy as np
import seaborn as sns
import sqlite3
import matplotlib.pyplot as plt

#enabling plotting of visusalizations in the notebook
%matplotlib inline

## a) Previewing the Database Tables
The database `im.db` has some datasets that will be useful for this analysis.

In [2]:
#creating a database connection
conn = sqlite3.connect('im.db')

#querying the database to view the database tables
q1 = """SELECT name
        FROM sqlite_master
        WHERE type = 'table';
     """
#executing the query using pandas
pd.read_sql(q1,conn)

,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


From the result above, we will need only the `movie_basics` and `movie_ratings` table. 
## Loading the two tables
Starting with `movie_basics`

In [3]:
#previewing 'movie_basics' by selecting all columns from the table
q2 = """SELECT * 
        FROM movie_basics;
     """
#executing the query and assigning to variable name movie_basics
movie_basics = pd.read_sql(q2,conn)

#loading the first five rows
movie_basics.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


Now `movie_ratings`

In [4]:
#selecting all columns 
q3 = """SELECT *
        FROM movie_ratings;
     """
#executing the query and assigning to variable name movie_ratings
movie_ratings = pd.read_sql(q3,conn)
movie_ratings.head()

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


## Joining the two tables using movie_id to create one table
Both tables have _movie_id_, we join them to form one table with more information.

In [5]:
#query the database to join the tables
# we'll use a LEFT JOIN 
q4 = """SELECT * 
        FROM movie_basics
        LEFT JOIN movie_ratings USING(movie_id);        
     """
#executing the query and assigning to variable name movies_info
movies_info = pd.read_sql(q4,conn)
movies_info.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43.0
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13.0
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0


In [6]:
#checking the info of this table
movies_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
 6   averagerating    73856 non-null   float64
 7   numvotes         73856 non-null   float64
dtypes: float64(3), int64(1), object(4)
memory usage: 8.9+ MB


## b) Loading the csv files
The csv files we'll use are `'tn.movie_budgets.csv'`, `'bom.movie_gross.csv'` and `'tmdb.movies.csv'`.

Previewing `'tn.movie_budgets.csv'` stored under data folder:

In [7]:
#using pandas to read the csv file and assigning it to variable name movie_budgets
movie_budgets = pd.read_csv('data/tn.movie_budgets.csv')

#loading the first five rows
movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [8]:
#checking the info about this file
movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


### Next `'bom.movie_gross.csv'`:

In [9]:
#reading and assigning to bom_movies
bom_movies = pd.read_csv('data/bom.movie_gross.csv')
bom_movies.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [10]:
#checking the info 
bom_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


### Finally `'tmdb.movies.csv'`.

In [11]:
#reading and assigning to tmdb_movies
tmdb_movies = pd.read_csv('data/tmdb.movies.csv')
tmdb_movies.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [12]:
#checking the info
tmdb_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB


## c) Data Cleaning
Some of the tables may contain duplicates, missing values or incorrect formatting. In this stage, we are going to clean each table to remove these duplicates or missing values present.
To start with, we'll first check for missing values in each table and deal with them.

## i) Identifying and handling missing values

### Missing values in `movies_info` 

In [13]:
#previewing the table again
movies_info.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43.0
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13.0
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0


In [14]:
#checking if missing values are present
#and counting how many they are in each column
movies_info.isnull().sum()

movie_id               0
primary_title          0
original_title        21
start_year             0
runtime_minutes    31739
genres              5408
averagerating      72288
numvotes           72288
dtype: int64

**Dropping all rows containing missing values**

In [15]:
#dropping rows with missing values
movies_info.dropna(inplace = True)

In [16]:
#checking again for missing values
movies_info.isnull().sum()

movie_id           0
primary_title      0
original_title     0
start_year         0
runtime_minutes    0
genres             0
averagerating      0
numvotes           0
dtype: int64

Missing values no longer exist.

In [17]:
#previewing the first 10 rows
movies_info.head(10)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43.0
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0
7,tt0137204,Joe Finds Grace,Joe Finds Grace,2017,83.0,"Adventure,Animation,Comedy",8.1,263.0
10,tt0146592,Pál Adrienn,Pál Adrienn,2010,136.0,Drama,6.8,451.0
11,tt0154039,So Much for Justice!,Oda az igazság,2010,100.0,History,4.6,64.0
12,tt0159369,Cooper and Hemingway: The True Gen,Cooper and Hemingway: The True Gen,2013,180.0,Documentary,7.6,53.0
13,tt0162942,Children of the Green Dragon,A zöld sárkány gyermekei,2010,89.0,Drama,6.9,120.0
14,tt0170651,T.G.M. - osvoboditel,T.G.M. - osvoboditel,2018,60.0,Documentary,7.5,6.0


## Creating a function to check and remove missing values.
To avoid repeating this process too many times with the remaining 3 dataframes, we'll define a function that will check if missing values are present in a dataframe and remove the rows with missing values.

In [18]:
#defining the function
def remove_missing_values(df):
    
    #checking for missing values
    missing_values = df.isnull().sum()
    
    #condition if the sum of missing values is greater than zero
    if missing_values.sum() > 0:
        print('Missing values found.')
        print(missing_values)
        
        #removing rows with missing values
        df = df.dropna(inplace=True)
        
        print('Rows with missing values removed!')
        
    else:
        #output if no missing values are present
        print('No missing values.')
        
    #return the dataframe    
    return df  

In [19]:
#testing our function with the table we removed missing values earlier
remove_missing_values(movies_info)

No missing values.


,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43.0
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0
7,tt0137204,Joe Finds Grace,Joe Finds Grace,2017,83.0,"Adventure,Animation,Comedy",8.1,263.0
...,...,...,...,...,...,...,...,...
146113,tt9911774,Padmavyuhathile Abhimanyu,Padmavyuhathile Abhimanyu,2019,130.0,Drama,8.4,365.0
146114,tt9913056,Swarm Season,Swarm Season,2019,86.0,Documentary,6.2,5.0
146115,tt9913084,Diabolik sono io,Diabolik sono io,2019,75.0,Documentary,6.2,6.0
146122,tt9914286,Sokagin Çocuklari,Sokagin Çocuklari,2019,98.0,"Drama,Family",8.7,136.0


### We use the function to check and remove missing values in table `movie_budgets`.

In [20]:
#checking and removing missing values
remove_missing_values(movie_budgets)

No missing values.


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


In [21]:
#previewing the info of the table
movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


### Removing missing values in `bom_movies`

In [22]:
#checking and removing missing values in table 'bom_movies'
remove_missing_values(bom_movies)

Missing values found.
title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64
Rows with missing values removed!


In [23]:
#previewing the info of the table
bom_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2007 entries, 0 to 3353
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           2007 non-null   object 
 1   studio          2007 non-null   object 
 2   domestic_gross  2007 non-null   float64
 3   foreign_gross   2007 non-null   object 
 4   year            2007 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 94.1+ KB


### Removing missing values in `tmdb_movies`

In [24]:
#checking and removing missing values in table 'bom_movies'
remove_missing_values(tmdb_movies)

No missing values.


,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186
...,...,...,...,...,...,...,...,...,...,...
26512,26512,"[27, 18]",488143,en,Laboratory Conditions,0.600,2018-10-13,Laboratory Conditions,0.0,1
26513,26513,"[18, 53]",485975,en,_EXHIBIT_84xxx_,0.600,2018-05-01,_EXHIBIT_84xxx_,0.0,1
26514,26514,"[14, 28, 12]",381231,en,The Last One,0.600,2018-10-01,The Last One,0.0,1
26515,26515,"[10751, 12, 28]",366854,en,Trailer Made,0.600,2018-06-22,Trailer Made,0.0,1


In [25]:
#previewing the info of the table
tmdb_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB


## ii) Identifying and handling duplicates
### Duplicates in `movies_info`

In [26]:
#checking for duplicates
movies_info.duplicated().value_counts()

False    65720
dtype: int64

`movies_info` does not appear to have duplicates. We then move to check if duplicates are present in `movies_budget`
### Duplicates in `movie_budget`

In [27]:
#checking for duplicates
movie_budgets.duplicated().value_counts()

False    5782
dtype: int64

`movie_budgets` does not appear to have duplicates. We then move to check if duplicates are present in `bom_movies`
### Duplicates in `bom_movies`

In [28]:
#checking for duplicates
bom_movies.duplicated().value_counts()

False    2007
dtype: int64

`bom_movies` does not appear to have duplicates. We then move to check if duplicates are present in `tmdb_movies`
### Duplicates in `tmdb_movies`

In [29]:
#checking for duplicates
tmdb_movies.duplicated().value_counts()

False    26517
dtype: int64

### iii) Formatting columns in `movie_budgets`

The data is now cleaned from having duplicates and missing values. However, the `movie_budgets` table has some of its columns in a format that may hinder our analysis. Here is a preview of the table:

In [30]:
#movie_budgets table
movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


The columns `'production_budget'`, `'domestic_gross'` and `'worldwide_gross'` have commas and the currency value `$` which will affect our analysis. We then have to clean this and convert it to `type = int`

### Defining a function to remove commas and dollar signs

In [31]:
#the function
def remove_commas_dollar_sign(df, column_names):
    for column_name in column_names:
        # Remove commas and dollar signs from the specified column
        df[column_name] = df[column_name].str.replace(',', '').str.replace('$', '')

        # Convert the column to integer data type
        df[column_name] = pd.to_numeric(df[column_name], errors='coerce').astype('Int64')

    return df

Applying the function to our table `movie_budgets`.

In [32]:
import warnings

# Ignore warning messages within the context
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    #renaming the table to cleaned_movie_budgets
    cleaned_movie_budgets = remove_commas_dollar_sign(movie_budgets, ['production_budget', 'domestic_gross', 'worldwide_gross'])

In [33]:
cleaned_movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747
